## Lab 6: Learning an NBC with EM

### Datasets load and split

In [112]:
import numpy as np
from sklearn import metrics, datasets
import scipy.stats as st

In [113]:
digits = datasets.load_digits()
digits_data = digits.data
digits_split = int(len(digits_data)*0.7)
x_train = digits_data[:digits_split]
x_test = digits_data[digits_split:]
digits_target = digits.target
y_train = digits_target[:digits_split]
y_test = digits_target[digits_split:]
print('Training data:', len(x_train), '\nTraining Labels:', len(y_train), '\nTesting Data:', 
      len(x_test), '\nTesting Labels:', len(y_test), '\nCheck:', 
      len(digits_data) == len(x_train) + len(x_test))
print(x_train.shape)
print(y_train.shape)
x_train /= 16
x_test /= 16

Training data: 1257 
Training Labels: 1257 
Testing Data: 540 
Testing Labels: 540 
Check: True
(1257, 64)
(1257,)


### EM Algorithm

#### Initialization

In [114]:
def initialize():
    indexes = np.random.randint(len(digits_data), size=int(len(digits_data)*0.1))
    theta = dict()
    temp = dict()
    for i in indexes:
        k = digits_target[i]
        pixels = digits_data[i]
        if k not in temp:
            temp[k] = list()
        temp[k].append(pixels)
        
    for k in temp:
        prior = 0.1
        values = np.array(temp[k])
        means = np.zeros(digits_data.shape[1])
        var = np.zeros(digits_data.shape[1])
        for i in range(len(values[0])):
            means[i] = np.mean(values[:,i])
            var[i] = np.var(values[:,i]) + 0.01
        theta[k] = np.array([prior, means, var])
    return theta

theta_init = initialize()
classes = 10

#### E-Step

In [115]:
def E_step(X, theta):
    r = np.zeros((X.shape[0],classes)) 
    for i in range(len(X)):
        prob = np.prod([st.norm.pdf(X[i], theta[k][1], np.sqrt(theta[k][2])) for k in range(classes)], axis = 1)
        prod = [theta[k][0]*prob[k] for k in range(classes)]
        den = np.sum(prod)
        r[i,:] = prod/den    
    return r

#### M-Step

In [116]:
def M_step(X, r):
    r_k = {k:np.sum(r[:,k]) for k in range(10)}
    theta = dict()
    for k in r_k:
        prior = r_k[k]/len(X)
        means = np.sum([r[i][k]*X[i] for i in range(len(X))], axis=0)/r_k[k]
        vars_ = np.sum([r[i][k]*(X[i]**2) for i in range(len(X))], axis=0)/r_k[k] - means**2 + 0.01
        theta[k] = np.array([prior, means, vars_])
    return theta

#### Termination

In [117]:
print(-1)
e = E_step(x_train, theta_init)
m = M_step(x_train, e)
for i in range(10):
    print(i)
    e = E_step(x_train, m)
    m = M_step(x_train, e)
#print(m)

-1
0
1
2
3
4
5
6
7
8
9


In [118]:
s = list()
for k in m:
    s.append(m[k][0])
    print( m[k][0])
print(s)

0.09864757352874674
0.0935270083628778
0.1038119285626458
0.1476850680954702
0.09318825061369254
0.07314633224486362
0.10023827897905312
0.11451624223726403
0.10646210189824976
0.0687772154771364
[0.09864757352874674, 0.0935270083628778, 0.1038119285626458, 0.1476850680954702, 0.09318825061369254, 0.07314633224486362, 0.10023827897905312, 0.11451624223726403, 0.10646210189824976, 0.0687772154771364]
